<a href="https://colab.research.google.com/github/Aktaruzzaman0/Sentiment-Analysis-useing-VADER/blob/main/Sentiment_Analysis_using_VADER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bluetooth-speakers-dataset-from-flipkart:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2874276%2F4956294%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240609%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240609T234512Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D161313d274e6a757eaa7ee552e62e5ee5469947c646d4699ef5be32b0a4d6a4b8072fa4a690e8af744040594153e949fa3055e6fefcc01a01f19ad67a760bc443427973664f3360b76e2ec1b2ac5c7df3e4b39b3df8c6a8d9e31cdc73dda7d71ded683e617cf5d8b6bc6876f939f6e6171d1d2072137c84fccf351a1e261f61487ab7ffe17d9dd7983f90332e65461be4c8b1b33b6c917fb3fa5781634f61fbcd76be57a0fa9aa2a30b39bfe0ae6b8bb75904c01c445e36ff08dc5ec221ebfaa87607f0156f5970299c4e85dd0d2cd85dcb6fa9ecdcf271a87f863dbd0688b978c63f37d0d07bbe22fbab34d3a0a4b9d65f1a49f1a4f13e5b3eddca37c34b6b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Multiclass classification using VADER module

## Import necessary modules
here, SentimentIntensityAnalyze is used to find out the polarity of the text, which is Positive, Negative, or Neutral.

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
import pandas as pd

## Load the dataset
Here, I attached the random dataset of one flipkart product called bluetooth speackers collected through web scraping.
- The dataset hass 12201 rows and 4 column.
- The column has product name, Rate, Review and Summary.

In [ ]:
df=pd.read_csv("/kaggle/input/bluetooth-speakers-dataset-from-flipkart/Bluetooth-speakers.csv",encoding="unicode_escape")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/flipkart-product-review-dataset/flipkart_product.csv'

#### We do sentiment based on review, you can also do based on summary.

In [ ]:
sentiments = SentimentIntensityAnalyzer()
df["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in df["Review"]]
df["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in df["Review"]]
df["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df["Review"]]
df['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in df["Review"]]
#df.head()

In [ ]:
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
df.head()

- Now we remove unnecessary column like scores of positive, negative, neutral and compound.

In [ ]:
df.drop(df.columns[[4, 5, 6, 7]], axis=1, inplace=True)
df

#### You can save your new .csv file through this command and download.

In [ ]:
df.to_csv('new_bluetooth-speakers.csv')

In [ ]:
df.shape

#### We count sentiment categories value

In [ ]:
df.Sentiment.value_counts()

So there are 9861 Positive, 1633 neutral and 888 negative values available.